### Citi Bike Ridership SARIMAX Forecast Build

[Looker Ridership Download](https://motivate.looker.com/explore/nyc/rental_details?qid=GMjolCqGOBTtP6CGgUMina)

[SARIMAX example](https://www.digitalocean.com/community/tutorials/a-guide-to-time-series-forecasting-with-arima-in-python-3)

In [3]:
from statsmodels import api as smi
from statsmodels import graphics as smg
from statsmodels import tsa as tsa 

import numpy as np
from scipy import stats as SPstats
from time import strptime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
# import pygal as pg
import itertools as it
import math
import datetime
%matplotlib inline

/Users/Sam_I_Am/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
rentalDat_initial = pd.read_csv('nyc rental_details 2018-05-19.csv')
rentalDat_initial.columns = ['Date', 'RentalCount']

rentalDat_initial['DoW'] = pd.to_datetime(rentalDat_initial['Date']).apply(datetime.date.weekday)
rentalDat_initial['MoY'] = pd.to_datetime(rentalDat_initial['Date']).apply(lambda x: x.month)
rentalDat_initial['LogRental'] = rentalDat_initial['RentalCount'].apply(math.log)
rentalDat_initial['Lag1 LogRental'] = rentalDat_initial['LogRental'].shift()
rentalDat_initial['Diff1 LogRental'] = rentalDat_initial['LogRental'] - rentalDat_initial['Lag1 LogRental']
rentalDat_initial['Date'] = pd.to_datetime(rentalDat_initial['Date'])
rentalDat_initial.sort_values('Date', inplace=True, ascending=False)


# set cutoff for startdate below
startDate = pd.to_datetime('2014-01-01')

rentalDat = rentalDat_initial[pd.to_datetime(rentalDat_initial['Date']) >= startDate]
rentalDat.set_index('Date', drop=False, inplace=True)

rentalDat.dropna(inplace=True)
rentalDat.shape

/Users/Sam_I_Am/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(1591, 7)

#### DeSeasonalize

In [5]:
DOWavg = pd.DataFrame(rentalDat.groupby('DoW')['LogRental'].mean())
DOWavg.columns = ['DoW avg']
MOYavg = pd.DataFrame(rentalDat.groupby('MoY')['LogRental'].mean())
MOYavg.columns = ['MoY avg']

rentalDat = rentalDat.merge(DOWavg, how='left', left_on='DoW', right_index=True)
rentalDat = rentalDat.merge(MOYavg, how='left', left_on='MoY', right_index=True)

rentalDat['LogRental_desea'] = rentalDat['LogRental'] - rentalDat['MoY avg'] - rentalDat['DoW avg']
rentalDat['LogRental_deseaMO'] = rentalDat['LogRental'] - rentalDat['MoY avg'] 

#### Select ARIMA (p, d, q)

In [ ]:
# use function below to calculate AICc for different combinations 

def calcAICc(observations, constant):
    
    trendVal = str(constant)
    # below is for all combinations of (p,q) <= 2
    combos = list(it.product(range(8),repeat=2))
    result_li = []
    
    
    for ea in combos: 
        ARMAaiccCalc = tsa.arima_model.ARMA(observations,order=ea) 
        # trend= indicates whether to use constant('c') or not ('nc'). 
        try:
            ARMAaiccCalc = ARMAaiccCalc.fit(trend=trendVal)
            logLikeli = ARMAaiccCalc.llf
            n_obs = ARMAaiccCalc.nobs
            #AICc calc
            AICc =  -2*logLikeli + 2*(sum(ea) + 1)*(n_obs/(n_obs-sum(ea)-2))
        except (ValueError, Exception): 
            AICc = 0
            pass

    
        result_li.append([ea, AICc])
        
    res_DF = pd.DataFrame(result_li)
    res_DF.columns = ['(p,q)','AICc']
    # res_DF['Abs AICc'] = abs(res_DF['AICc'])
    res_DF.sort_values('AICc', ascending=True, inplace=True)
    
    return res_DF
        



In [ ]:
# run AICc both with and without constant added
aicsNC = calcAICc(rentalDat['LogRental_deseaMO'],'nc')
aicsC = calcAICc(rentalDat['LogRental_deseaMO'],'c')

aicsNC['Constant'] = 'NC'
aicsC['Constant'] = 'C'
allAICC = pd.concat([aicsNC, aicsC])

allAICC[allAICC['AICc'] != 0].sort_values('AICc')

#### Fit ARMA(5,5) Model

In [ ]:
# fit an ARMA(5,5) model with constant. 
rentalDeSea_ARMA55 = tsa.arima_model.ARMA(rentalDat['LogRental_deseaMO'],order=(5,5))
# trend='nc' removes constant 
rentalDeSea_ARMA55 = rentalDeSea_ARMA55.fit(trend='c')
rentalDeSea_ARMA55.summary()

#### Select Seasonal (P, D, Q, S)

In [6]:
p = d = q = range(0, 8)

# Generate all different combinations of p, q and q triplets
pdq = list(it.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 7) for x in list(it.product(p, d, q))]

In [ ]:
pdqTest = seasonal_pdq[5]
(pdqTest[0] + pdqTest[3] )

In [7]:
# use function below to calculate AICc for different Seasonal PDQS combinations 

def calcSARIMA_AICc(observations, seasonalPeriod):
    
    s = int(seasonalPeriod)
    nonSeasonalOrd = (5,0,5)
    p = d = q = range(0, 3)

    # Generate all different combinations of p, q and q triplets
    pdq = list(it.product(p, d, q))

    # Generate all different combinations of seasonal p, q and q triplets
    seasonal_pdq = [(x[0], x[1], x[2], s) for x in list(it.product(p, d, q))]
    

    result_li = []
    
    
    for ea in seasonal_pdq: 
        SARMAaiccCalc = tsa.statespace.sarimax.SARIMAX(observations, 
                                                       order=nonSeasonalOrd, 
                                                      seasonal_order=(ea[0],ea[1],ea[2],ea[3]), 
                                                      enforce_stationarity=False, 
                                                      enforce_invertibility=False)

        try:
            SARMAaiccCalc = SARMAaiccCalc.fit()
            logLikeli = SARMAaiccCalc.llf
            n_obs = SARMAaiccCalc.nobs
            #AICc calc
            AICc =  n_obs*logLikeli + 2*(nonSeasonalOrd[0] + nonSeasonalOrd[2] + ea[0] + ea[2] + 1)*(n_obs/(n_obs - nonSeasonalOrd[0] - nonSeasonalOrd[2] - ea[0] - ea[2] - 2))
        except (ValueError, Exception): 
            AICc = 0
            pass

    
        result_li.append([ea, AICc])
        
    res_DF = pd.DataFrame(result_li)
    res_DF.columns = ['(P,D,Q,S)','AICc']
    # res_DF['Abs AICc'] = abs(res_DF['AICc'])
    res_DF.sort_values('AICc', ascending=True, inplace=True)
    
    return res_DF
        




In [8]:
calcSARIMA_AICc(rentalDat['LogRental_deseaMO'], 7)

/Users/Sam_I_Am/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/Sam_I_Am/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/Sam_I_Am/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/Sam_I_Am/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/Sam_I_Am/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimizat

,"(P,D,Q,S)",AICc
6,"(0, 2, 0, 7)",-2.410155e+06
15,"(1, 2, 0, 7)",-1.952924e+06
24,"(2, 2, 0, 7)",-1.759628e+06
7,"(0, 2, 1, 7)",-1.429248e+06
16,"(1, 2, 1, 7)",-1.404336e+06
3,"(0, 1, 0, 7)",-1.392690e+06
25,"(2, 2, 1, 7)",-1.343037e+06
12,"(1, 1, 0, 7)",-1.310801e+06
21,"(2, 1, 0, 7)",-1.300219e+06
0,"(0, 0, 0, 7)",-1.131378e+06


In [ ]:
SARMAaiccCalc = tsa.statespace.sarimax.SARIMAX(rentalDat['LogRental_deseaMO'], order=(5,0,5), 
                                                      seasonal_order=(pdqTest[0],pdqTest[1],pdqTest[2],pdqTest[3]), 
                                                      enforce_stationarity=False, 
                                                      enforce_invertibility=False)
SARMAaiccCalc = SARMAaiccCalc.fit()
logLikeli = SARMAaiccCalc.llf
n_obs = SARMAaiccCalc.nobs
#AICc calc
AICc =  -2*logLikeli + 2*(sum(ea) + 1)*(n_obs/(n_obs-sum(ea)-2))

In [ ]:
AICc